# ブロックチェーンの構造

## ハッシュ関数

ハッシュ関数は、入力された数値を一定の規則に則った数値に変換する関数で、以下のような特徴がある

1. 不可逆性
    - 一方向にしか計算できず、逆算ができない
2. 機密性
    - 入力データが少しでも変われば、出力データは大きく変わる
3. 固定長データ
    - 入力データの長さを問わず、出力データは同じ長さのデータとなる
4. 高処理速度
    - 入力データから出力データを簡単に計算できる

ハッシュ関数の持つ不可逆的性質は、第3者に知られたくない情報を隠すために利用される

また、入力データの長さを問わずを問わず、出力値のデータ長が常に一定になるため、多くのデータを要約してデータサイズを圧縮することにも利用される

### ハッシュ関数の種類

| ハッシュ関数 | 詳細 |
| :-- | :-- |
| SHA-256 | Secure Hash Algorism 256 bit の略で、入力データのサイズにかかわらず256ビットのハッシュ値を生成する。ブロックチェーン技術全般で広く使われており、SHA-256を2回重ね掛けするような使い方もよく見られる |
| RIPEMD-160 | RACE Integrity Primitives Evaluation Message Digest の略で、160ビットのハッシュ値を生成する。SHA-256よりも小さいサイズのハッシュ値を生成するため、データサイズを節約したい場合に利用される |
| HMAC-SHA516 | キーとデータのペアを入力値として512ビットのハッシュ値を生成する。階層的決定性ウォレット（HDウォレット）で利用される |

In [1]:
# Julia 標準ライブラリの SHA で SHA-256 ハッシュ化が可能
using SHA

sha256("hello") |> bytes2hex

"2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824"

In [2]:
# 入力値が少し異なるだけで出力値が大きく変わる
sha256("hello") |> bytes2hex |> println
sha256("hallo") |> bytes2hex |> println

2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
d3751d33f9cd5049c4af2b462735457e4d3baf130bcbb87f389e349fbaeb20b9


In [3]:
# 入力値のサイズにかかわらず常に同じ長さのハッシュ値を出力する
sha256("hello") |> bytes2hex |> println
sha256("Hello, World !") |> bytes2hex |> println
sha256("SHA-256ハッシュアルゴリズムは常に256ビット長のハッシュ値を生成する") |> bytes2hex |> println

2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
7d486915b914332bb5730fd772223e8b276919e51edca2de0f82c5fc1bce7eb5
ce077bf893417e3622768aa174236d81c4520e433c379a5cb530c8839c71b7eb


## ブロックの構造

### ブロックの内部構造
ブロックチェーンのブロックの中身は以下のような構造になっている

| property | 型 | 値 |
| :-- | :-- | :-- |
| magicnum | `UInt32` | マジックナンバー (ブロック識別子) |
| blocksize | `UInt32` | ブロックサイズ |
| blockheader | `BlockHeader` | ブロックヘッダ情報 |
| Txsvi | `UInt32` | 取引の数 |
| Txs | `Vector{Transaction}` | 取引リスト |

### ブロックヘッダ構造
ブロックヘッダは以下のような情報を含む80バイトのデータである

| property | サイズ | 値 |
| :-- | :-- | :-- |
| Version | 4バイト | ビットフィールド |
| Prev block hash | 32バイト | 1つ前のブロックのハッシュ値 |
| Merkleroot | 32バイト | 取引を要約したハッシュ値 |
| Time | 4バイト | ブロックが生成された時間を示すタイムスタンプ |
| Bits | 4バイト | マイニングの難易度 |
| Nonce | 4バイト | マイニングで条件を満たしたナンス値 |

ブロックチェーンは、次のブロックのブロックヘッダに1つ前のブロックのハッシュ値が取り込まれ（Prev block hash）、鎖のように繋げられていく

このように、ハッシュ関数を用いて次々とデータを鎖状に繋いでいく構造のことを **ハッシュチェーン** と呼ぶ

### トランザクション
マイニングを経て取り込まれるトランザクション（取引）の数は多くて3000ほどである

これらのトランザクションはリストとして格納されており、その数によっては大きなデータサイズとなる場合もある

基本的には取引のリストを格納するだけだが、依存関係のある取引の場合には、その格納順序が重要になる場合もある

こういった場合、トランザクションの順番に関する情報までやり取りしなければならなくなり、マイニングが非効率になる

そこで CTOR (Canonical Transaction Ordering Rule) 方式という、取引のハッシュ値を昇順で並べるといった方式も採用されている

トランザクションは **マークルツリー (Markletree)** というデータ構造を用いてハッシュ化し、**マークルルート (Markleroot)** として要約される

マークルルートはそのブロックのブロックヘッダに格納され、トランザクションが正しいものか検証する際に用いられる

```bash
# Markletree

Markleroot
 |_ H1234
 |   |_ H12
 |   |   |_ H1 <- Tx1
 |   |   |_ H2 <- Tx2
 |   |
 |   |_ H34
 |       |_ H3 <- Tx3
 |       |_ H4 <- Tx4
 |
 |_ H5677
     |_ H56
     |   |_ H5 <- Tx5
     |   |_ H6 <- Tx6
     |
     |_ H77
         |_ H7 <- Tx7
         |_ H7 <- (Tx7)
```